In [32]:
from dotenv import load_dotenv
import os
import json
from openai import OpenAI
import pinecone
from pinecone import Pinecone, ServerlessSpec

load_dotenv()


True

In [33]:
# pinecone index - one seperate db inside of your pinconce instance
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# pc.create_index(
#     # were doing 1536 dimensions because that s the number of dimensions open ai has
#     # if you use a free model , you have to make sure the dimensions match from the embeddings and pinecone
#     # metric is used for searching, its a method of searching
#     name="rag", dimension= 1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"),
# )



In [34]:
import json
data=json.load(open("lreviews.json"))
data["reviews"]

[{'professor': 'Dr. John Smith',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Excellent professor! Very knowledgeable and approachable.'},
 {'professor': 'Dr. Emily Johnson',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Great teacher, but the exams were quite challenging.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Average teaching style, but helpful during office hours.'},
 {'professor': 'Dr. Sarah Davis',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Amazing lectures and very supportive!'},
 {'professor': 'Dr. David Wilson',
  'subject': 'Biology',
  'stars': 2,
  'review': 'Not very engaging and the material was hard to follow.'},
 {'professor': 'Dr. Laura Martinez',
  'subject': 'History',
  'stars': 4,
  'review': 'Interesting lectures, but a lot of reading assignments.'},
 {'professor': 'Dr. James Anderson',
  'subject': 'Philosophy',
  'stars': 5,
  'review': 'Thought-provoking and inspiring!'},
 {'pro

In [35]:
processed_data = []
client = OpenAI()

# embeddings assign everything a number. Kinda insane but this is how they search. they instead of trying to read the strings, just look at which numbers are closer to each other. Its like translating a string into numbers so that they can be compared
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review["review"],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })

In [36]:
processed_data[0]

{'values': [-0.01197446882724762,
  -0.06290911138057709,
  -0.06475035846233368,
  0.030968673527240753,
  -0.009346862323582172,
  -0.007518406491726637,
  -0.030968673527240753,
  0.04124894365668297,
  -0.04306461289525032,
  -0.01750458963215351,
  0.023552559316158295,
  -0.026314422488212585,
  0.013707026839256287,
  -0.0030559508595615625,
  0.003417166881263256,
  -0.0024821609258651733,
  0.009071954526007175,
  0.010887624695897102,
  0.035546205937862396,
  0.03902411088347435,
  0.019307471811771393,
  -0.028436966240406036,
  0.008873765356838703,
  0.02250407449901104,
  -0.027132753282785416,
  -0.028948422521352768,
  0.02512528747320175,
  0.018207840621471405,
  0.00986471213400364,
  0.022759802639484406,
  0.09543772786855698,
  -0.007914785295724869,
  -0.03321908041834831,
  -0.025342656299471855,
  -0.04452226310968399,
  0.03774546831846237,
  -0.0063772196881473064,
  0.029204150661826134,
  -0.013310648500919342,
  0.018847161903977394,
  0.03700385615229606

In [38]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1",
    )

{'upserted_count': 20}

In [39]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}